# Finding Eigenpairs of the DtN map via Minimising the Rayleigh Quotient

For the duration of this script, we let $\Omega=\left[0,1\right)^2$, fix $N\in\mathbb{N}$, $\theta\in\left[-\pi,\pi\right)^2$, and $\omega^2>0$.
Our goal is to determine the first $N$ eigenvalues $\lambda_n$ and eigenfunctions $\varphi_n$ of the Dirichlet-to-Neumann operator $\mathcal{D}$;
\begin{align*}
    \mathrm{dom}\mathcal{D} &= \left\{ (g,h)\in L^2(\partial\Omega)\times L^2(\partial\Omega) \ \middle\vert \ \exists v\in H^2(\Omega) \text{ s.t. } (\Delta_{\theta}-\omega^2)v = 0, \ v\vert_{\partial\Omega}=g, \ \nabla^{\theta}v\cdot n\vert_{\partial\Omega} = h \right\}, \\
    \mathcal{D}g &= h.
\end{align*}

We will do this by minimising the Rayleigh quotient, that is by solving
\begin{align*}
    \varphi_n &= \mathop{\mathrm{argmin}}_{\varphi\perp\varphi_1,...,\varphi_{n-1}} \left\{ \frac{\lvert\lvert \nabla^{\theta}\varphi \rvert\rvert_{L^2(\Omega)} - \omega^2 \lvert\lvert \varphi \rvert\rvert_{L^2(\Omega)} }{ \lvert\lvert \varphi \rvert\rvert_{L^2(\partial\Omega)} } \right\}, \\
    \lambda_n &= \min_{\varphi\perp\varphi_1,...,\varphi_{n-1}} \left\{ \frac{\lvert\lvert \nabla^{\theta}\varphi \rvert\rvert_{L^2(\Omega)} - \omega^2 \lvert\lvert \varphi \rvert\rvert_{L^2(\Omega)} }{ \lvert\lvert \varphi \rvert\rvert_{L^2(\partial\Omega)} } \right\}.
\end{align*}
Clearly, it is only necessary for us to determine $\varphi_n$ and then evaluate the Rayleigh quotient to obtain $\lambda_n$.

To solve this problem, we will treat it as a contrained optimisation problem.
In order to approximate our functions $\varphi_n$, we will use a truncated Fourier basis
\begin{align*}
    \varphi &= \sum_{\alpha,\beta\in\mathbb{Z}}^{\lvert\alpha+\beta\rvert\leq M} c_{\alpha\beta}\mathrm{e}^{2\pi\mathrm{i}(\alpha x + \beta y)}.
\end{align*}
Some quick calculations show that
\begin{align*}
    \lvert\lvert \nabla^{\theta}\varphi \rvert\rvert_{L^2(\Omega)}
    &= \sum_{\alpha,\beta\in\mathbb{Z}}^{\lvert\alpha+\beta\rvert\leq M} 
    \lvert c_{\alpha\beta}\rvert^2
    \lvert \theta + 2\pi\begin{pmatrix} \alpha \\ \beta \end{pmatrix} \rvert^2, \\
    \lvert\lvert \varphi \rvert\rvert_{L^2(\Omega)}
    &= \sum_{\alpha,\beta\in\mathbb{Z}}^{\lvert\alpha+\beta\rvert\leq M} 
    \lvert c_{\alpha\beta}\rvert^2, \\
    \lvert\lvert \varphi \rvert\rvert_{L^2(\partial\Omega)}^2
    &= 4\sum_{\alpha,\beta\in\mathbb{Z}}^{\lvert\alpha+\beta\rvert\leq M} \lvert c_{\alpha\beta}\rvert^2, \\
    \langle \varphi, \varphi_k \rangle &=
    \sum_{\alpha,\beta\in\mathbb{Z}}^{\lvert\alpha+\beta\rvert\leq M} c_{\alpha\beta}\overline{c}_{\alpha\beta}^{(k)}.
\end{align*}
This means we reduce the optimisation problem above to (given $1\leq n\leq N$):
\begin{align*}
    \text{Minimise } J(\mathbf{c}) &= \sum_{\alpha,\beta\in\mathbb{Z}}^{\lvert\alpha+\beta\rvert\leq M}
    \lvert c_{\alpha\beta}^{(n)}\rvert^2
    \left[ \lvert \theta + 2\pi\begin{pmatrix} \alpha \\ \beta \end{pmatrix} \rvert^2 - \omega^2 \right]
    \quad &\text{w.r.t } \mathbf{c} = \left(c_{\alpha\beta}^{(n)}\right)\in\mathbb{C}^{(2M+1)^2}, \\
    \text{Subject to } \sum_{\alpha,\beta\in\mathbb{Z}}^{\lvert\alpha+\beta\rvert\leq M} \lvert c_{\alpha\beta}^{(n)}\rvert^2 &= \frac{1}{4},
    \quad & \\
    \sum_{\alpha,\beta\in\mathbb{Z}}^{\lvert\alpha+\beta\rvert\leq M} c_{\alpha\beta}^{(n)}\overline{c}_{\alpha\beta}^{(k)} &= 0,
    \quad & \text{for } 1\leq k\leq n-1.
\end{align*}
We solve these sequentially; first for $n=1$, and the solution can then be used in the constraints for the next problem when determining $n=2$.
The eigenvalues can be computed after each minimisation by evaluating the objective function $J$ at the found solution.
Note that $J$ is real-valued, and thus the eigenvalues are going to be real.
For each $n$, the problem involves finding $(2M+1)^2$ (complex) unknowns that are contrained by $n$ conditions.

In [1]:
import numpy as np
from numpy import exp, pi

import scipy as sp
from scipy.optimize import minimize

#### `J(c, theta, omega)`:

Evaluates the functional $J(\mathbf{c})$ at the given values for the quasimomentum $\theta$ and $\omega$;
\begin{align*}
    J(\mathbf{c}) &= \sum_{\alpha,\beta\in\mathbb{Z}}^{\lvert\alpha+\beta\rvert\leq M}
    \lvert c_{\alpha\beta}^{(n)}\rvert^2
    \left[ \lvert \theta + 2\pi\begin{pmatrix} \alpha \\ \beta \end{pmatrix} \rvert^2 - \omega^2 \right]
\end{align*}
where the argument $\mathbf{c} = \left(c_{\alpha\beta}^{(n)}\right)\in\mathbb{C}^{(2M+1)^2}$, as above.

In [2]:
def J(c, theta, omega):
    '''
    Evaluates the objective function J.
    INPUTS:
        c: (2M+1)^2 complex, vector of the c_ab^n (\mathbf{c} above)
        theta: (2,) float, value of the quasi-momentum
        omega: float, value of omega
    OUTPUTS:
        Jval: float, value of the objective function J
    '''
    
    M = (np.sqrt(c.shape[0]) - 1) // 2
    sqCoeffs = np.abs(c)*np.abs(c)
    
    alpha = beta = np.arange(-M, M+1)
    # this is a ((2M+1)**2,2) numpy array of all combinations of alpha, beta that we need to use,
    # these are stacked by [a0, b0], [a0, b1], ..., [a0, B(2M+1)], [a1, b0] etc, IE as \mathbf{c} is.
    abVals = np.array(np.meshgrid(alpha, beta)).T.reshape(-1,2)
    # now compute the theta - 2\pi (a,b) values.... again as an ((2M+1)**2,2) array
    tMinusIndex = theta.reshape((1,2)) - 2*pi*abVals
    # now compute |theta - 2\pi (a,b)|^2 - \omega^2, as an ((2M+1)**2,) array
    prods = np.linalg.norm(tMinusIndex, axis=1)**2 - omega*omega
    # it should now just be a case of a sum of element-wise vector products
    Jval = np.sum(sqCoeffs * prods)
    return Jval

### `FourierFunction` class

We will need a framework for storing our eigenfunctions and their associated eigenvalues, to do this we will create a Python class, `FourierFunction`.
This class will have attributes:
- $\theta$ (`theta`): The value of the quasimomentum.
- $\omega$ (`omega`): The value of $\omega$.
- $M$ (`M`): The value of $M$ in the description above, we will truncate the Fourier series at indices of order $M$.
- $\lambda$ (`lbda`): The eigenvalue associated to the eigenfunction.
- `cMat`: The coefficients of this function's Fourier series, in matrix form
- `cVec`: The coefficients of this function's Fourier series, in vector form
and methods
- `val`: Evaluates the function $\varphi$ stored in this class instance at the point $x=(x_1,x_2)$, vectorised.

It will be necessary for us to store the Fourier coefficients in both a matrix and vector form.
The matrix form (`cMat`) is natural, as we can simply store the coefficients by `cMat[a+M,b+M]` = $c_{(a)(b)}$.
Vector form is less convenient, but we can make use of the reshape method to create `cVec` as
\begin{align*}
    \mathbf{c} = \begin{pmatrix} c_{(-M)(-M)} \\ c_{(-M)(-M+1)} \\ \vdots \\ c_{(-M)(2M)} \\ c_{(-M+1)(-M)} \\ c_{(-M+1)(-M+1)} \\ \vdots \end{pmatrix},
\end{align*}
and running `cVec.resize((2M+1,2M+1))` will then produce an $2M+1\times 2M+1$ matrix whose $ij$-th entry is $c_{(i)(j)}$.
To this end, we can translate as follows;
\begin{align*}
    c_{(i)(j)} = \mathbf{c}[(j+M) + (2M+1)*(i+M)],
\end{align*}
for any $i,j\in\{-M,...,M\}$.

#### `__init__(self, theta, omega, c)`: (Initialisation method)
Creates an instance of the class, setting the attributes of the instance.
$\theta$ and $\omega$ are set automatically to the values passed in.
The values of $M$ and the appropriate `cVec` and `cMat` are deduced from $\mathbf{c}$.
$\lambda$ is computed using the functional $J(\mathbf{c})$ and the given $\theta, \omega$ values.

The input $\mathbf{c}$ must be passed in as either a vector or matrix containing the values $c_{\alpha\beta}$, from this the class will auto-detect the value of $M$ and create a view of $\mathbf{c}$ in the other format.


#### `val(self, x)`:
Evaulates the function $\varphi$ stored in this class instance, at the point(s) $x=(x_1,x_2)$.
Is vectorised, so $x$ should be of shape $(l,2)$ for some $l\geq 1$.
We then evaluate the series
\begin{align*}
    \varphi &= \sum_{\alpha,\beta\in\mathbb{Z}}^{\lvert\alpha+\beta\rvert\leq M} c_{\alpha\beta}\mathrm{e}^{2\pi\mathrm{i}(\alpha x + \beta y)}
    = \sum_{\alpha,\beta\in\mathbb{Z}}^{\lvert\alpha+\beta\rvert\leq M} c_{\alpha\beta}\mathrm{e}^{2\pi\mathrm{i}\alpha x}\mathrm{e}^{2\pi\mathrm{i}\beta y},
\end{align*}
returning a vector of length $l$ containing the values which were computed.

In [3]:
# create a class to handle evaluation and storage of our eigenfunctions
class FourierFunction:
    '''
    ATRIBUTES:
        theta
        omega
        M
        lbda
        cMat
        cVec
    METHODS:
        eval
        set_lambda
    '''
    
    def __init__(self, theta, omega, c):
        '''
        Create an instance with the Fourier coefficients passed in the matrix or vector c.
        INPUTS:
            theta: (2,) float, value of the quasimomentum.
            omega: float, value of omega.
            c: (2M+1,2M+1) complex double, the Fourier coefficients of this function. 
            If c is of shape  ((2M+1)^2,), then this is interpretted column-wise, as above.
        OUTPUTS:
            FourierFunction instance, with theta, omega, M, cMat, cVec, and lbda all set.
        '''
        
        self.theta = theta
        self.omega = omega
        if c.ndim==2:
            # passed cMat
            self.cMat = c
            self.M = (c.shape[0]-1) // 2 #-1 is actually superflous here, but it doesn't hurt...
            self.cVec = c.reshape((c.shape[0]*c.shape[0],)) #create a view rather than a whole new matrix
        elif c.ndim==1:
            # passed cVec
            self.cVec = c
            self.M = (np.sqrt(c.shape[0]) - 1) // 2 #-1 is actually superflous here, but it doesn't hurt
            self.cMat = c.reshape((2*M+1,2*M+1))
        else:
            raise ValueError('Unexpected number of dimensions in c array, got %d, expected 1 or 2' % c.ndim)
        self.set_lambda() 
        return
    
    def val(self, x):
        '''
        Evaluates the FourierFunction at the point(s) x
        INPUTS:
            x: (l,2) float, (x,y) coordinate pairs, stored in each column of the array.
        OUTPUTS:
            xVals: (l,) complex, values of the FourierFunction at the input x.
        '''
        
        mInts = np.arange(-self.M, self.M+1)
        if x.ndim==2:
            xVals = np.zeros(x.shape[0], dtype=complex)
            # it's a for loop because I can't wrap 3D vectorisation around my head
            for l in range(x.shape[0]):           
                expAX = exp(2.j*pi*x[l,0]*mInts)
                expBY = exp(2.j*pi*x[l,1]*mInts)
                # expAX[i] = e^{2i\pi x[l]*(i-M)}, similarly for expBY
                expMat = np.outer(expAX, expBY)
                # np.outer(a,b) returns M_{i,j} = a_i * b_j, thus
                # expMat[i,j] = e^{2i\pi x[l]*(i-M)} * e^{2i\pi y[l]*(j-M)}
                # now we multiply by the Fourier coefficients...
                fTerms = expMat * self.cMat
                # then sum all the terms in the matrix!
                xVals[l] = np.copy(fTerms.sum())
        elif x.ndim==1:           
            expAX = exp(2.j*pi*x[0]*mInts)
            expBY = exp(2.j*pi*x[1]*mInts)
            # expAX[i] = e^{2i\pi x[l]*(i-M)}, similarly for expBY
            expMat = np.outer(expAX, expBY)
            # np.outer(a,b) returns M_{i,j} = a_i * b_j, thus
            # expMat[i,j] = e^{2i\pi x[l]*(i-M)} * e^{2i\pi y[l]*(j-M)}
            # now we multiply by the Fourier coefficients...
            fTerms = expMat * self.cMat
            # then sum all the terms in the matrix!
            xVals = fTerms.sum()
        else:
            raise ValueError('Unexpected dimension of x, got %d, expected 1 or 2', x.ndim)
        return xVals
    
    def set_lambda(self):
        '''
        Sets the value of lambda to be that of the functional J(c), using c = self.cVec.
        If we have solved the corresponding minimisation problem for the vector self.cVec, this will set lambda to
        be the eigenvalue corresponding to this eigenfunction.
        '''
        self.lbda = J(self.cVec, self.theta, self.omega)
        return

## Setting up the Optimisation Problem

The function $J$ can now be evaluated, so we just need to setup the constraints on this problem to proceed.
However, one hiccup in `SciPy`'s routines is that it cannot deal with complex arguments when solving a minimisation problem, see [this StackExchange article](https://stackoverflow.com/questions/51211055/can-scipy-optimize-minimize-functions-of-complex-variables-at-all-and-how).

As such, we have to use wrappers for the real and imaginary parts of each of the $c_{\alpha\beta}^{(n)}$, optimise this, and then interpret the result.
Our wrappers adopt the following convention; for an incoming vector $z$ of $n$ complex numbers, we create the vector $x$ via the equality $z(j) = x(2j) + \mathrm{i}x(2j+1)$.
Similarly, given a real vector $x$ containing $2n$ values, we construct $n$ complex numbers in an array $z$ via the same formula.

#### `Real2Comp(x)`
Given a vector of floats $x$ of length $2n$, return the array of complex numbers $z$ defined by `z[j] = x[2j] + i*x[2j+1]`.

#### `Comp2Real(x)`
Given a vector of complex numbers $z$ of length $n$, return the array $x$ of the real and imaginary parts defined by `z[j] = x[2j] + i*x[2j+1]`.

In [4]:
def Real2Comp(x):
    '''
    Given a vector x of length (2n,), which stores the real and imaginary parts of complex numbers z as
    z[j] = x[2j] + i*x[2j+1], return the vector z.
    INPUTS:
        x: (2n,) float, real and imaginary parts of a vector of complex numbers z
    OUTPUTS:
        z: (n,) complex, z[j] = x[2j] + i*x[2j+1]
    '''
    
    z = np.zeros((len(x)//2,), dtype=complex)
    z = x[np.arange(0, len(x), 2)] + 1.j*x[np.arange(1, len(x), 2)]
    return z

def Comp2Real(z):
    '''
    Given a vector z of length (n,) of complex numbers, return a real array x where
    z[j] = x[2j] + i*x[2j+1]
    '''
    
    x = np.zeros((2*len(z),), dtype=float)
    x[np.arange(0, len(x), 2)] = np.real(z)
    x[np.arange(1, len(x), 2)] = np.imag(z)
    return x

Now we need to setup our optimisation problem.
We will need to wrap our functional $J$ so that it can take $\mathbf{c}$ as a vector of real values containing the real and imaginary parts.
The constraint
\begin{align*}
    \frac{1}{4}
    &= \sum_{\alpha,\beta\in\mathbb{Z}}^{\lvert\alpha+\beta\rvert\leq M} \lvert c_{\alpha\beta}^{(n)}\rvert^2 
    = \sum_{\alpha,\beta\in\mathbb{Z}}^{\lvert\alpha+\beta\rvert\leq M} \Re\left(c_{\alpha\beta}^{(n)}\right)^2  + \Im\left(c_{\alpha\beta}^{(n)}\right)^2
\end{align*}
can simply be computed by `np.sum(cc**2)=0.25`, provided `cc = Comp2Real(c)`.
How many _more_ constraints we have depends on $n$, which is annoying since it will require us to redefine our function handles for the constraint, it's Jacobian, and it's Hessian combination each time we want to find another eigenfunction.
Nonetheless, these other constraints can be written as
\begin{align*}
    0
    &= \sum_{\alpha,\beta\in\mathbb{Z}}^{\lvert\alpha+\beta\rvert\leq M} c_{\alpha\beta}^{(n)}\overline{c}_{\alpha\beta}^{(k)}
    = \sum_{\alpha,\beta\in\mathbb{Z}}^{\lvert\alpha+\beta\rvert\leq M} \Re\left(c_{\alpha\beta}^{(n)}\right)\Re\left(c_{\alpha\beta}^{(k)}\right) + \Im\left(c_{\alpha\beta}^{(n)}\right)\Im\left(c_{\alpha\beta}^{(k)}\right) - \mathrm{i}\Re\left(c_{\alpha\beta}^{(n)}\right)\Im\left(c_{\alpha\beta}^{(k)}\right) + \mathrm{i}\Im\left(c_{\alpha\beta}^{(n)}\right)\Re\left(c_{\alpha\beta}^{(k)}\right)
\end{align*}
for $1\leq k\leq n-1$.
Again, `SciPy` can only handle _real_ constraints, so we have to split this constraint into two constraints for the real and imaginary parts,
\begin{align*}
    0 &= \sum_{\alpha,\beta\in\mathbb{Z}}^{\lvert\alpha+\beta\rvert\leq M} \Re\left(c_{\alpha\beta}^{(n)}\right)\Re\left(c_{\alpha\beta}^{(k)}\right) + \Im\left(c_{\alpha\beta}^{(n)}\right)\Im\left(c_{\alpha\beta}^{(k)}\right), \\
    0 &= \sum_{\alpha,\beta\in\mathbb{Z}}^{\lvert\alpha+\beta\rvert\leq M} \Im\left(c_{\alpha\beta}^{(n)}\right)\Re\left(c_{\alpha\beta}^{(k)}\right) - \Re\left(c_{\alpha\beta}^{(n)}\right)\Im\left(c_{\alpha\beta}^{(k)}\right). 
\end{align*}
The former can be computed via `cc_n[0::2]*cc_k[0::2] + cc_n[1::2]*cc_k[1::2]` (using `start:stop:step`), where `cc_n` represents $\mathbf{c}^{(n)}$ and `cc_k` are the known coefficients for the function $\varphi_k$.
The latter sum can be computed in a similar fashion, being equal to `cc_n[1::2]*cc_k[0::2] - cc_n[0::2]*cc_k[1::2]`.

In [ ]:
theta = np.zeros((2,))
omega = 0.

# we can now pass cc in as a float array, and still have J operate on it.
Jopt = lambda cc: J(Real2Comp(cc), theta, omega)
# now we must specify the constraints, remember that we are also using cc and not simply c!

In [6]:
theta = np.zeros((2,))
omega = 0.

cVals = np.zeros((3,3), dtype=complex)
cVals[0,1] += 1.0
cVals[1,2] += 1.0
cVals[2,2] += 1.0

xy = np.zeros((5,2), dtype=float)
xy[1,:] = np.ones((1,2), dtype=float) / 4.
xy[2,:] = 0.5*np.ones((1,2), dtype=float)
xy[3,:] = (3/4)*np.ones((1,2), dtype=float)
xy[4,:] = np.ones((1,2), dtype=float)

f = FourierFunction(theta, omega, cVals)